<a href="https://colab.research.google.com/github/Indrakaran1411/Product_sales_analysis_Dashboard/blob/main/Ola_Ride_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'chh-ola:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F15848%2F670358%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240611%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240611T150716Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D807a4949a81d3ab2fb65b864734feddcd6a2cd28ab1e3005401877742980bbf140ca3da455cf0aae2ba8ccfa26e370bc803f0a43f019f0cfd79cf2605ab39203b231f6d474af27760960b5060c7fac504761957b4607d64b37d6857a4587fc47e20571bd3307bee98bbe72f5e4ac87a84b6dba5284c6ab09d2f5fcb65fac433fef18dab09f50a3e32682e986792ffaabdcc0327f39fe8f343c6f6673f478aa072b73153403434202ab6654d495b5fe800170131ecc6a8bcdab89dd962a6eaf8bcd5e068db6b8353f9b7486f3ca48b6f0350a349168bbcfcacffcb74a0a8c5cb245c52aaedf297e648616e691ffee3d70ba24d0d96bcf4c6ab0f923c01c9eadd1'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 29099188 bytes downloaded
Downloaded and uncompressed: chh-ola
Data source import complete.


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
from sklearn import metrics
from sklearn.model_selection import train_test_split
import warnings; warnings.simplefilter('ignore')

In [ ]:
data=pd.read_csv("../input/chh-ola/train.csv")
test=pd.read_csv("../input/chh-ola/test.csv")

In [ ]:
data.isnull().sum()

ID                        0
vendor+AF8-id             0
pickup+AF8-loc            1
drop+AF8-loc              1
driver+AF8-tip            2
mta+AF8-tax               1
distance                  1
pickup+AF8-time           1
drop+AF8-time             1
num+AF8-passengers        1
toll+AF8-amount           2
payment+AF8-method        1
rate+AF8-code             1
stored+AF8-flag           1
extra+AF8-charges         1
improvement+AF8-charge    2
total+AF8-amount          2
dtype: int64

In [ ]:
data.columns

Index(['ID', 'vendor+AF8-id', 'pickup+AF8-loc', 'drop+AF8-loc',
       'driver+AF8-tip', 'mta+AF8-tax', 'distance', 'pickup+AF8-time',
       'drop+AF8-time', 'num+AF8-passengers', 'toll+AF8-amount',
       'payment+AF8-method', 'rate+AF8-code', 'stored+AF8-flag',
       'extra+AF8-charges', 'improvement+AF8-charge', 'total+AF8-amount'],
      dtype='object')

In [ ]:
data.rename(columns = {'vendor+AF8-id': "vendor_id",
                        'pickup+AF8-loc': "pickup_loc",
                        'drop+AF8-loc' : "drop_loc",
                        'driver+AF8-tip': "driver_tip",
                        'mta+AF8-tax' : "mta_tax",
                        'pickup+AF8-time' : "pickup_time",
                        'drop+AF8-time' : "drop_time",
                        'num+AF8-passengers' : "num_passengers",
                        'toll+AF8-amount' : "toll_amount",
                        'payment+AF8-method' : "payment_method",
                        'rate+AF8-code' : "rate_code",
                        'stored+AF8-flag' : "stored_flag",
                        'extra+AF8-charges' : "extra_charges",
                        'improvement+AF8-charge' : "improvement_charge",
                        'total+AF8-amount': "total_amount"
                        }, inplace = True)

In [ ]:
data.shape

In [ ]:
data['pickup_time']=pd.to_datetime(data['pickup_time'])

In [ ]:
data['drop_time']=pd.to_datetime(data['drop_time'])

In [ ]:
data['trip_duration']=data['drop_time']-data['pickup_time']

In [ ]:
data['trip_duration'] = data['trip_duration'].astype('timedelta64[s]')

In [ ]:
data.drop(['pickup_time', "drop_time",'ID', "vendor_id", "drop_loc", "pickup_loc", "stored_flag", "mta_tax", "improvement_charge" ], axis = 1, inplace = True)

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
data.isnull().sum()

In [ ]:
data.isnull().sum().sum()

In [ ]:
cat_cols = ["driver_tip", "toll_amount", "extra_charges", "total_amount"]
num_cols = [c for c in data.columns if c not in cat_cols]
print (num_cols)

In [ ]:
for c in num_cols :
  data[c].fillna((data[c].mean()),inplace=True)

NameError: name 'num_cols' is not defined

In [ ]:
data.isnull().sum()

ID                    0
vendor_id             0
pickup_loc            1
drop_loc              1
driver_tip            2
mta_tax               1
distance              1
pickup_time           1
drop_time             1
num_passengers        1
toll_amount           2
payment_method        1
rate_code             1
stored_flag           1
extra_charges         1
improvement_charge    2
total_amount          2
dtype: int64

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 17 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   ID                  1048575 non-null  int64  
 1   vendor_id           1048575 non-null  object 
 2   pickup_loc          1048574 non-null  float64
 3   drop_loc            1048574 non-null  float64
 4   driver_tip          1048573 non-null  object 
 5   mta_tax             1048574 non-null  object 
 6   distance            1048574 non-null  float64
 7   pickup_time         1048574 non-null  object 
 8   drop_time           1048574 non-null  object 
 9   num_passengers      1048574 non-null  float64
 10  toll_amount         1048573 non-null  object 
 11  payment_method      1048574 non-null  float64
 12  rate_code           1048574 non-null  float64
 13  stored_flag         1048574 non-null  object 
 14  extra_charges       1048574 non-null  object 
 15  improvement_cha

In [ ]:
for c in cat_cols :
  data[c] = pd.to_numeric(data[c], errors='coerce')

NameError: name 'cat_cols' is not defined

In [ ]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 17 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   ID                  1048575 non-null  int64  
 1   vendor_id           1048575 non-null  object 
 2   pickup_loc          1048574 non-null  float64
 3   drop_loc            1048574 non-null  float64
 4   driver_tip          1048573 non-null  object 
 5   mta_tax             1048574 non-null  object 
 6   distance            1048574 non-null  float64
 7   pickup_time         1048574 non-null  object 
 8   drop_time           1048574 non-null  object 
 9   num_passengers      1048574 non-null  float64
 10  toll_amount         1048573 non-null  object 
 11  payment_method      1048574 non-null  float64
 12  rate_code           1048574 non-null  float64
 13  stored_flag         1048574 non-null  object 
 14  extra_charges       1048574 non-null  object 
 15  improvement_cha

In [ ]:
data.isnull().sum().sum()

In [ ]:
data.dropna(inplace= True)

In [ ]:
data.shape

(1048573, 17)

In [ ]:
y=data['total_amount']

In [ ]:
data.drop(["total_amount"], axis=1, inplace = True)

In [ ]:
data.shape

In [ ]:
from xgboost import XGBRegressor
train_x,test_x,train_y,test_y=train_test_split(data,y,test_size=.075,random_state=42)

In [ ]:
model = XGBRegressor(max_depth=8, n_estimators = 750, learning_rate= 0.025,random_state=42)
model.fit(train_x,train_y)

In [ ]:
model.score(train_x,train_y)

In [ ]:
model.score(test_x,test_y)

In [ ]:
test=pd.read_csv("../input/chh-ola/test.csv")

In [ ]:
test['pickup_time']=pd.to_datetime(test['pickup_time'])
test['drop_time']=pd.to_datetime(test['drop_time'])
test['trip_duration']=test['drop_time']-test['pickup_time']
test['trip_duration'] = test['trip_duration'].astype('timedelta64[s]')

In [ ]:
test.shape

In [ ]:
test.info()

In [ ]:
test.drop(['pickup_time', "drop_time",'ID', "vendor_id", "drop_loc", "pickup_loc", "stored_flag", "mta_tax", "improvement_charge" ], axis = 1, inplace = True)

In [ ]:
test.shape

In [ ]:
test.info()

In [ ]:
predict = model.predict(test)

In [ ]:
ID = np.arange(154235)

In [ ]:
len(ID)

In [ ]:
df = pd.DataFrame({"ID" : ID,
                   "total_amount" : predict}, columns = ["ID", "total_amount"]).set_index("ID")

In [ ]:
df.head()